# Aggregating and Cleaning Baltimore City Government Salaries for Fiscal Years 2011-2019

Baltimore City government publishes [open data](https://data.baltimorecity.gov/browse?category=City+Government) about their full-time or civil service employee's salary, hire data, and earned income for fiscal years 2011-2019 (Baltimore City government's fiscal year runs from July 1-June 30, where the ending year of each cycle denotes the "fiscal year"). While this provides transpareny on these type of employees and some city government salary spending--this doesn't include salaries for contract employees--it is challenging to perform long-term trend analysis on this data because different years' exported data have different column headers, different labels for department names, and different labels for employee names. This notebook aims to clean and organize this data to make it easier to draw long-term comparisons that relate to salary earnings, salary offers, paid position types, and Baltimore City government tenure.

## import libraries

In [3]:
# for data analysis
import pandas as pd 
import numpy as np

# get today's date
from datetime import date, datetime

# to create visualizations 
import plotly.express as px

# access local file
import glob
import os


## about the data

Each Baltimore City Employee Salary csv export on Baltimore Open Data contains the following column headers in the same order: 

 - __name__: Baltimore City government employee's name (last, first)
 - __jobtitle__: employee's job title classification 
 - __deptid__: employee's ID within their department
 - __descr__: employee's current department/agency where they are employed
 - __hire_dt__: date employee was initially hired by Baltimore City government (not necessarily the date they were hired for their current position or their current agency)
 - __annual_rt__: employee's annual salary as stated in their employment contract
 - __gross__: employee's gross earned income for the fiscal year; this may be more than their annual salary if they earn overtime, etc., or it may be less than their annual income if they don't work their full 40 hour/week

Although this is clear and relatively consistent across all fiscal years, each fiscal year data upload has different formatting for each fiscal year (e.g. name vs Name vs NAME), which make it difficult to easily concatenate data with pandas. 

## import data

In these functions and loops, we reformat the salary data in all of the 2011-2019 fiscal years so that:
 - All of the data has consistent columns (we also rename "name" to "employee_name" and "descr" to "department_name"
 - There is an added "fiscal_year" column to differentiate the fiscal year for those earnings
 - The annual and gross salary columns are formatted as floats (or, essentially, workable numbers to perform caluculations)
 - The hire date column is reformatted to a datetime object so that we can perform datetime functions which allows us to
 - Create an additional column ("years_in_gov") that calculates time the employee has worked for city government
 - We reformat the department names ("descr" / "dept_name") since some fiscal years define department names as only the name and some department names include a sub-department code as a number in parentheses. This makes it difficult to look at long-term department specific trends

In [4]:
# functions to load in and reformat the salary data so that we can work with all of the data over all of the years available
# read in all fiscal year csv file and add column for fiscal year to differentiate

def fiscal_year(csv):
    
    # define the path for the file we want to extract the fiscal year from
    filename = os.path.basename(csv)
    
    # replace "." in path name with "_"
    # split on "_" and keep the 5th value as the date
    fy = filename.replace(".", "_").split("_")[4]
    
    # make sure fiscal year is a number
    # keep last two digits of the fiscal year term
    # add 20 to make it "2011", etc.
    fy = "20" + fy[-2:]
    
    return fy

In [5]:
# for loop to read in all files, reformat headers, add in fiscal year column, and concatenate with other csvs

# initiate data empty data frame to append edited dataframes to later
df_bmore_salary = pd.DataFrame([])

# for each file in the file in this path name (all downloaded files for Baltimore City gov salaries)
for counter, file in enumerate(glob.glob("bmore_salaries_2011_2018/*.csv")):
    
    # load the file from bmore_salaries_folder
    df_salary = pd.read_csv(file)
    
    # rename the column headers so that all years are consistent
    
    # new column names
    salary_col = ['employee_name', 'jobtitle', 'deptid', 'dept_name', 'hire_dt', 'annual_rt', 'gross']
    
    # replace original column headers with new salary_col names
    df_salary.columns = salary_col
        
    #Call function to extract the date from the filename
    fy_date = fiscal_year(file)
    
    #Add the date string to the dataframe before appending
    df_salary['fiscal_year'] = fy_date
        
    #Stitch it all together
    df_bmore_salary = df_bmore_salary.append(df_salary, sort = False)
    


In [6]:
# function to transform annual and gross salaries into floats

def salary_number_formatting(df):
    # convert annual_rt column to float
    df["annual_rt"] = df["annual_rt"].astype(float)
    # convert gross to float
    df["gross"] = df["gross"].astype(float)
    
    # convert hire date to datetime
    df["hire_dt"] = pd.to_datetime(df["hire_dt"])
    # calculate tenure in city government and add new column for this value
    df["years_in_gov"] = (datetime.now() - df["hire_dt"])/np.timedelta64(1,"Y")

In [7]:
# use salary_number_formatting function for dataframe
salary_number_formatting(df_bmore_salary)

In [8]:
# edit the department name to remove the department name code in parentheses

df_bmore_salary["dept_name"] = df_bmore_salary["dept_name"].str.split("(", n = 1, expand = True)

In [10]:
df_bmore_salary.head()

,employee_name,jobtitle,deptid,dept_name,hire_dt,annual_rt,gross,fiscal_year,years_in_gov
0,"Aaron,Patricia G",Facilities/Office Services II,A03031,OED-Employment Dev,1979-10-24,50845.0,45505.94,2011,40.062412
1,"Aaron,Petra L",ASSISTANT STATE'S ATTORNEY,A29005,States Attorneys Office,2006-09-25,56595.0,51588.89,2011,13.140573
2,"Abaineh,Yohannes T",EPIDEMIOLOGIST,A65026,HLTH-Health Department,2009-07-23,56500.0,50633.26,2011,10.315053
3,"Abdal-Rahim,Naim A",EMT Firefighter,A64063,Fire Academy Recruits,2011-03-30,33476.0,3888.95,2011,8.631240
4,"Abdi,Ezekiel W",POLICE OFFICER,A99398,Police Department,2007-06-14,50919.0,51421.73,2011,12.423241


In [15]:
# how many unique department names
df_bmore_salary["dept_name"].nunique()

141

In [16]:
# how many unique job titles
df_bmore_salary["jobtitle"].nunique()

2418

In [19]:
# how many fiscal years does this cover?
df_bmore_salary["fiscal_year"].nunique()

9

In [20]:
# high-level information about the new dataframe
df_bmore_salary.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 137433 entries, 0 to 13482
Data columns (total 9 columns):
employee_name    137433 non-null object
jobtitle         137433 non-null object
deptid           137433 non-null object
dept_name        137433 non-null object
hire_dt          137123 non-null datetime64[ns]
annual_rt        137433 non-null float64
gross            128954 non-null float64
fiscal_year      137433 non-null object
years_in_gov     137123 non-null float64
dtypes: datetime64[ns](1), float64(3), object(5)
memory usage: 10.5+ MB


## export data in csv to use in additional analysis

In [13]:
# export as a csv, but don't include index as a column
df_bmore_salary.to_csv("total_bmore_city_salaries_2011_2019.csv", index = False)

In [14]:
# export as an excel file, but don't include index as a column
df_bmore_salary.to_excel("total_bmore_city_salaries_2011_2019.xlsx", index = False)